## Convolving light-curves 

The script convolves ligth curves to get the microlensing signal.

Rewritten by: Soumya Shreeram <br>
Script originally written by: Eric Paic <br>
Date: 02nd March 2020 <br>

In [ ]:
import numpy as np
from astropy.io import fits
import os,sys
from astropy.convolution import convolve_fft
import scipy.signal as ss
#from guppy import hpy

### 1. Setting the path for the directories and input parameters

In [ ]:
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir)
print("Does the directory exist? \n>",os.path.isdir(data_dir))

# setting the paths
datadir = data_dir + '/Data/'
scriptdir = data_dir + '/Data/scripts/'
resultdir = data_dir + '/Data/results/'
mapdir = data_dir + '/Data/maps/convolved/'
storagedir = data_dir + '/Data/maps/storage/'

All input parameters:

In [ ]:
list_r0 = [2,4,10,15,20,30,40,60,80,100]

list_img = ['A1','B1','A2','B2','A3','B3','A4','B4']

list_comb = [('A1', 'B2'),('A2','B1'),('A3','B3'),('A4','B3'),('A2','B4'),('A1','B3'),('A6','B2'),('A6','B3'),('A7','B3'),('A8','B2'),('A8','B3')]

einstein_r = 3.414e16
cm_per_pxl = (20*einstein_r)/8192
ld_per_pxl = 30000000000*3600*24/cm_per_pxl

dim = 512

In [ ]:
def mapDiff(mapA,mapB,r0,comb):
    """
    Function takes the logarithmic difference between the two maps A and B
    Input:
    @mapA, mapB :: two randomly chosen maps
    @r0 :: scale radius
    @comb :: combination of the two maps eg. A1, B2 or A3, B2, etc.
    """
    img = fits.open(mapA)[0]
    map_A = img.data[:, :]

    img = fits.open(mapB)[0]
    map_B = img.data[:, :]

    final_map = map_A/map_B

    hdu = fits.PrimaryHDU(final_map)
    os.chdir(datadir)
    hdu.writeto(resultdir+'/map%s-%s_fml09_R%s_thin_disk.fits'%(comb[0],comb[1],r0))
    return final_map

The models of the light source used to convolve the mgnification maps are:
* `thin_disk` (default option) more information available [here](https://arxiv.org/pdf/1707.01908.pdf).
* `sersic`
* `thin_disk&node`
* `wavy_hole`
* `wavy`
* `sersic`

In [ ]:
def calR(x, xc, y, yc):
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def calXi(r, Rin, R0):
    """
    Function checks the value of r and calculates xi accordingly
    """
    if r<Rin:
        return 0
    else:
        if Rin == 0:
            xi = (r / R0) ** (3. / 4.) * (1) ** (-1. / 4.)
        else:
            xi = (r/R0)**(3./4.)*(1-np.sqrt(Rin/r))**(-1./4.)
    return xi

def getprofilevalue(x, y, xc, yc, xn, yn, I0, reff_pix=0.2, sersic_index=4.0):
    """
    Function to get the profile values for each particular choice of model 
    """
    if model == "thin_disk":		
        r = calR(x, xc, y, yc)
        xi = calXi(r, Rin, R0)
        profile_val = I0/(np.exp(xi)-1)

	if model == "thin_disk&node":
		r = calR(x, xn, y, yn)
        xi = calXi(r, Rin, R0)
        profile_val = I0/(np.exp(xi)-1) + I0/(np.exp((r/(R0*0.4106))**(3/4))-1)

	if model =="wavy_hole":
		r = calR(x, xc, y, yc)
        n = 3
        beta = n*np.pi/(2*R0)
        # calculates the profile value based on conditions on beta*r
        if beta*r < 2*np.pi:
            profile_val = I0*beta/(n*np.pi**2)*np.power(np.sin(beta*r),2)/r
        else :
            profile_val = 0

	if model =="wavy":
		r = calR(x, xc, y, yc)
        n = 3
        beta = n*np.pi/(2*R0)
        if beta*r < np.pi/2:
            profile_val = I0*beta/(n*np.pi**2)
        elif beta * r < 2 * np.pi and beta * r > np.pi/2:
            profile_val = I0*beta/(n*np.pi**2)*np.power(np.sin(beta*r),2)
            
    if model == "sersic":
		r = calR(x, xc, y, yc)
        profile_val = I0 * np.exp(-(r / reff_pix) ** (1.0 / sersic_index))
    return profile_val

def generateDataFiles(new_canvas.fits):
    """
    Function to generate the data files
    """
    os.system('cp new_canvas.fits toconv.fits')
    os.system('cp new_canvas.fits source_profile.fits')  # non-convolved profile, 128*128
    os.system('cp new_canvas.fits gaussian.fits')  # gaussian, 128*128

    profile = fits.open(data_dir+'source_profile.fits', mode='update')[0]
    gaussian = fits.open(data_dir+'gaussian.fits', mode='update')[0]
    toconv = fits.open(data_dir+'toconv.fits', mode='update')[0]

    data = toconv.data[:,:]
    pdata = profile.data[:,:]
    gdata = gaussian.data[:,:]
    return data, pdata, gdata

def get2ddiracvalue(x,y,xc,yc):
    """
    Function to get the 2D dirac values to fill the gaussian ???
    """
    if x == xc and y ==yc:
        return 1
    else:
        return 0
    
def outputMapParams(img):
    map_d = img.data[:, :]
	macro_mag = np.mean(map_d)
	map_d = map_d / macro_mag
	return map_d, macro_mag

def convolve(R0, map_name, model="thin_disk", Rin =0, I0=1):
    """
    Input:
    @map_name :: fits file of the magnification map
    @model :: the model of the light source that the magnification map will be convoluted with.
    @R0 :: float; scale radius, for thin_disk, R0 is R0 (~10^14 cm for 1131 & 0435), for sersic R0 is fwhm. (Units: pixels)
    @Rin :: float. Useful only for thin_disk
    @I0 :: float. Useful for both models
    #create source profile

    """
    img_name = map_name.split('map')[2].split('.')[0]
	img = fits.open(map_name)[0]
	map_d, macro_mag  = outputMapParams(img)
    
	xc, yc, xn, yn = 256, 256, 128, 128
	
	# generates the profile, gaussian, and toconv fits files from new_canvas.fits
    data, pdata, gdata = generateDataFiles(new_canvas.fits)
    
    # fills the data arrays based of the choice of the model
    for lind, line in enumerate(gdata):
        for cind, elt in enumerate(line):
            gdata[lind][cind] = get2ddiracvalue(cind+1, lind+1, 256, 256)
            pdata[lind][cind] = getprofilevalue(x, y, xc, yc, xn, yn, I0, reff_pix=0.2, sersic_index=4.0)
            
    # convolves the data
    output = convolve_fft(gdata, pdata)

    for lind, line in enumerate(data):
        for cind, elt in enumerate(line):
            data[lind][cind] = out[lind][cind]
    # ???
    out2 = ss.fftconvolve(map_d, data, mode="valid")
    hdu = fits.PrimaryHDU(out2)

    hdu.writeto(storagedir+'/convolved_map_%s_fft_%s_%i_fml09.fits'%(img_name,model,R0))
    return

In [ ]:
hdu = fits.PrimaryHDU(np.zeros((dim, dim)))

os.chdir("../")
for img_n in list_img:

    for r0 in list_r0:
        print(r0)
        convolve(r0, mapdir+"/map%s.fits"%(img_n),"thin_disk")

for comb in list_comb:

    for elem in list_r0:
        print(elem)
        final_map = mapDiff(storagedir+"/convolved_map_%s_fft_thin_disk_%s_fml09.fits"%(comb[0], lem),storagedir+"/convolved_map_%s_fft_thin_disk_%s_fml09.fits"%(comb[1],elem), elem, comb)